# Python API for KiFMM-rs

We provide a full, object oriented, Python API for our Rust library. Below are a number of examples of how to achieve different functionality.

In [1]:
%gui qt
import numpy as np
from mayavi import mlab
import matplotlib.pyplot as plt

from kifmm_py import (
    KiFmm,
    LaplaceKernel,
    SingleNodeTree,
    EvalType,
    BlasFieldTranslation,
    FftFieldTranslation,
)

********************************************************************************
         to build the TVTK classes (9.2). This may cause problems.
         Please rebuild TVTK.
********************************************************************************



## Setup

Setting up an FMM requires some coordinate data associated with source and target particles, which we expect as a flat Fortran ordered buffer of shape (n_coordinates, 3), where each column corresponds to the component of each axis.

Additionally the user needs to choose some basic parameters, principally

## Kernel

We support both `LaplaceKernel` and `HelmholtzKernel`, which has been tested to low-frequencies.

## Field Translation

Users must specify the acceleration schemes for the multipole to local (M2L) field translation step of the FMM.

We currently offer `FftFieldTranslation`, and `BlasFieldTranslation`, where the matrices associated with `BlasFieldTranslation` can optionally be compressed with a randomised SVD, or a deterministic SVD.

## Tree

Only `SingleNodeTree`s are provided at present, users can optionally discretise with a specified critical value of particles per leaf box `n_crit`, or by explicitly setting the tree's `depth`. In the latter case `expansion_order` must be set for each level of the tree, allowing for the option to set variable expansion order by level. This can be efficacious for the `HelmholtzKernel`.


## Basic Example (Laplace FMM + FFT M2L)

In [2]:
np.random.seed(0)

dim = 3
dtype = np.float32
ctype = np.complex64

# Set FMM Parameters
expansion_order = np.array([6], np.uint64)  # Single expansion order as using n_crit
n_vec = 1
n_crit = 150
n_sources = 1000
n_targets = 2000

# Setup source/target/charge data in Fortran order
sources = np.random.rand(n_sources * dim).astype(dtype)
targets = np.random.rand(n_targets * dim).astype(dtype)
charges = np.random.rand(n_sources * n_vec).astype(dtype)
 
# EvalType computes either potentials (EvalType.Value) or potentials + derivatives (EvalType.ValueDeriv)
kernel = LaplaceKernel(
    dtype, EvalType.Value
) 

field_translation = FftFieldTranslation(
    kernel, block_size=32
)  
tree = SingleNodeTree(sources, targets, charges, n_crit=n_crit, prune_empty=True)

# Create FMM runtime object
fmm = KiFmm(expansion_order, tree, field_translation, timed=True)

# Evaluate potentials
fmm.evaluate()

# Examine potentials
fmm.all_potentials

array([[67.89123, 67.05206, 67.09231, ..., 62.72009, 79.68142, 65.05178]],
      dtype=float32)

## Compare with Direct Evaluation

Note, as a part of the FMM input coordinate data is sorted into a `Morton' defined order, as a part of tree construction. Therefore to compare with direct evaluation, the evaluated potentials must be unpermuted as a post processing step

In [3]:
# Unsort into input order
fmm.unsort_all_potentials()

# Examine unsorted potentials
fmm.all_potentials_r

# Direct evaluation
direct = np.zeros(n_targets).astype(dtype)
fmm.evaluate_kernel(EvalType.Value, sources, targets, charges, direct)


assert np.allclose(fmm.all_potentials_r, direct)

## 3D Plot with MayaVi

In [5]:
from kifmm_py import read_stl_triangle_mesh_vertices

In [6]:
(sources, faces) = read_stl_triangle_mesh_vertices("example.stl")
x = sources[:, 0]
y = sources[:, 1]
z = sources[:, 2]
sources = sources.flatten()

#### Plot Data

In [7]:
mlab.view(azimuth=40, elevation=70, distance="auto", focalpoint="auto")
fig = mlab.figure(
    size=(960, 1080), bgcolor=(1, 1, 1)
)  # This sets the window size for rendering
plot = mlab.triangular_mesh(
    x, y, z, faces, color=(0.5, 0.5, 0.5), representation="surface", figure=fig
)
mlab.show()

#### Setup FMM and Evaluate

In [8]:
dim = 3
dtype = np.float32

# Set FMM Parameters
expansion_order = np.array([6])
n_vec = 1
n_crit = 150
n_sources = len(sources) // dim

# Set Random charges
charges = np.random.rand(n_sources).astype(dtype)

kernel = LaplaceKernel(
    dtype, EvalType.Value
) 

field_translation = FftFieldTranslation(
    kernel, block_size=32
)  
tree = SingleNodeTree(sources, sources, charges, n_crit=n_crit, prune_empty=True)

# Create FMM runtime object
fmm = KiFmm(expansion_order, tree, field_translation, timed=True)

fmm.evaluate()
fmm.unsort_all_potentials()

#### Plot Solution

In [9]:
mlab.view(azimuth=40, elevation=70, distance="auto", focalpoint="auto")
fig = mlab.figure(
    size=(960, 1080), bgcolor=(1, 1, 1)
)  # This sets the window size for rendering
solution = mlab.triangular_mesh(
    x, y, z, faces, scalars=np.log(fmm.all_potentials_r).flatten(), representation="surface"
)